# <center> Lab 3-1: Test Receive CAN
## <center> ENGR 580A2: Secure Vehicle and Industrial Networking
## <center><img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="600" />
### <center> Instructor: Dr. Jeremy Daily<br>Fall 2020

## Prerequisites
Login tip: Send a keepalive signal every 60 seconds from the connection menu of PuTTy.

This notebook must be run on a Linux machine with SocketCAN enabled.
```
sudo ip link add dev vcan1 type vcan
sudo ip link set vcan1 up
```

If you are running from PuTTy through Windows, then be sure to start the Jupyter server with the following commands:

```
jupyter notebook --ip cybertruck1.engr.colostate.edu --no-browser
```

or for the Beagle Bone Black connected by USB:
```
jupyter notebook --ip 192.168.7.2 --no-browser
```
Once these commands are successful, there will be a tokenize URL to paste into your browser. Note: this is not a secure connection, so be sure you are behind a firewall.


Be sure can-utils is installed:
```
sudo apt install can-utils
```

Here is a CAN trace to examine:

https://www.engr.colostate.edu/~jdaily/J1939/files/candump_DDEC_key_off-on_engine_running_request_on.txt

Let's add the CAN trace to the home directory:

```
wget https://www.engr.colostate.edu/~jdaily/J1939/files/candump_DDEC_key_off-on_engine_running_request_on.txt
```

Add a virtual can interface:

```
sudo ip link add dev vcan1 type vcan
sudo ip link set vcan1 up
```

Set canplayer to play an infinite loop of the CAN log in the background. Once this is going, you can logout of the machine and the vcan1 interface will still have data.

```
canplayer -l i -I candump_DDEC_key_off-on_engine_running_request_on.txt vcan1=can1 &
```

Ideally, the canplayer command will be run from a screen that can be detached from an SSH session. 

https://linoxide.com/linux-how-to/keep-processes-running-ssh-logout-linux/

## Reference Material
from https://www.kernel.org/doc/html/latest/networking/can.html#the-virtual-can-driver-vcan 
```
struct can_frame {
        canid_t can_id;  /* 32 bit CAN_ID + EFF/RTR/ERR flags */
        __u8    can_dlc; /* frame payload length in byte (0 .. 8) */
        __u8    __pad;   /* padding */
        __u8    __res0;  /* reserved / padding */
        __u8    __res1;  /* reserved / padding */
        __u8    data[8] __attribute__((aligned(8)));
};
```

Copy and modify the code from here to injest CAN data.
https://www.engr.colostate.edu/~jdaily/J1939/tools.html



In [1]:
#!/usr/bin/python3
import socket
import struct
import time

# Open a socket and bind to it from SocketCAN
sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)
interface = "vcan0"

# Bind to the interface
sock.bind((interface,))

# To match this data structure, the following struct format can be used:
can_frame_format = "<lB3x8s"

# Write date to a local log file.
with open("candump_python.log", 'w') as outfile:
    for i in range(25):
        can_packet = sock.recv(16)
        can_id, can_dlc, can_data = struct.unpack(can_frame_format, can_packet)
        extended_frame = bool(can_id & socket.CAN_EFF_FLAG)
        if extended_frame:
            can_id &= socket.CAN_EFF_MASK
            can_id_string = "{:08X}".format(can_id)
        else: #Standard Frame
            can_id &= socket.CAN_SFF_MASK
            can_id_string = "{:03X}".format(can_id)
        hex_data_string = ' '.join(["{:02X}".format(b) for b in can_data[:can_dlc]])
        print("{} {} [{}] {}".format(interface, can_id_string, can_dlc, hex_data_string))
        outfile.write("({:12.6f}) {} {}#{}\n".format(time.time(),
                                                     interface, 
                                                     can_id_string, 
                                                     hex_data_string.replace(" ","")))

vcan0 18F0000F [8] F0 7D FF FF 0F 7D 00 19
vcan0 18FEF200 [8] 2D 00 2E 0A 0D 06 00 FF
vcan0 18FEE000 [8] FF FF FF FF 54 97 5E 00
vcan0 0CF00400 [8] 70 7D 83 46 14 00 F0 FF
vcan0 08FE6E0B [8] E0 09 80 09 C0 09 60 09
vcan0 0CF00400 [8] 70 7D 83 46 14 00 F0 FF
vcan0 0CF00400 [8] 70 7D 83 40 14 00 F0 FF
vcan0 18F0010B [8] C0 FF F0 FF FF 0D FF FF
vcan0 08FE6E0B [8] 00 0A 40 09 C0 09 60 09
vcan0 0CF00331 [8] FF FF FF FF FF FF FF FF
vcan0 0CF00400 [8] 70 7D 83 40 14 00 F0 FF
vcan0 18FEBF0B [8] C0 09 81 79 7B 73 7D 77
vcan0 0CF00400 [8] 70 7D 83 40 14 00 F0 FF
vcan0 0CF00300 [8] F1 00 0B FF FF FC FF FF
vcan0 18FEBF0B [8] 70 09 84 76 80 78 84 7A
vcan0 08FE6E0B [8] E0 09 00 09 A0 09 20 09
vcan0 0CF00400 [8] 70 7D 83 46 14 00 F0 FF
vcan0 0CF00400 [8] 70 7D 83 46 14 00 F0 FF
vcan0 08FE6E0B [8] 00 0A 20 09 A0 09 00 09
vcan0 0CF00400 [8] 70 7D 83 46 14 00 F0 FF
vcan0 18F00131 [8] FF FF FF 3F 00 FF FF FF
vcan0 18FEF131 [8] F3 FF FF 57 CC FF FF FF
vcan0 18FEF031 [8] FF FF FF FF FF FF FF FF
vcan0 18FEF

The messages shown above should change based on where we are in the drive cycle. If there is no output and the indicator says In \[*], then can socket is blocking because there is no data. 